In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
apps = pd.read_csv('./data/apps.tsv', delimiter='\t',encoding='utf-8')
user_history = pd.read_csv('./data/user_history.tsv', delimiter='\t',encoding='utf-8')
jobs = pd.read_csv('./data/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
users = pd.read_csv('./data/users.tsv' ,delimiter='\t',encoding='utf-8')

b'Skipping line 122433: expected 11 fields, saw 12\n'
b'Skipping line 602576: expected 11 fields, saw 12\n'
b'Skipping line 990950: expected 11 fields, saw 12\n'
/home/nghiatruong/.conda/envs/science/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
apps = apps[apps.Split == 'Train'].drop(['WindowID', 'Split'], axis=1)
apps.isnull().sum()

UserID             0
ApplicationDate    0
JobID              0
dtype: int64

In [4]:
user_history = user_history[user_history.Split == 'Train'].drop(['WindowID', 'Split', 'Sequence'], axis=1).dropna()
user_history.isnull().sum()

UserID      0
JobTitle    0
dtype: int64

In [5]:
users = (
    users[(users.Split == 'Train') & (users.Country == 'US')]
    .drop(['ZipCode',
           'GraduationDate',
           'WorkHistoryCount',
           'CurrentlyEmployed',
           'ManagedOthers',
           'ManagedHowMany',
           'WindowID',
           'Split'],
          axis=1)
    .fillna('Not Applicable')
)
users.isnull().sum()

UserID                  0
City                    0
State                   0
Country                 0
DegreeType              0
Major                   0
TotalYearsExperience    0
dtype: int64

In [6]:
jobs = jobs[jobs.Country == 'US'].drop(['WindowID', 'Zip5'], axis=1).fillna('Not Applicable')
jobs.isnull().sum()

JobID           0
Title           0
Description     0
Requirements    0
City            0
State           0
Country         0
StartDate       0
EndDate         0
dtype: int64

In [7]:
uids = (apps.groupby('UserID')
            .size()
            .reset_index(name='AppCount')
            .sort_values('AppCount', ascending=False)[:10000]
            .UserID.values)
users_10k = users[users.UserID.isin(uids)].copy()

In [8]:
jids = (apps.groupby('JobID')
            .size()
            .reset_index(name='AppCount')
            .sort_values('AppCount', ascending=False)[:10000]
            .JobID.values)
jobs_10k = jobs[jobs.JobID.isin(jids)].copy()

In [9]:
apps_10k = apps[apps.UserID.isin(uids) & apps.JobID.isin(jids)].copy()

In [10]:
user_history_10k = user_history[user_history.UserID.isin(uids)].copy()
user_history_10k['AllTitles'] = user_history_10k.groupby('UserID').transform(', '.join)
user_history_10k.drop(['JobTitle'], axis=1, inplace=True)
user_history_10k.drop_duplicates(inplace=True)
users_10k = users_10k.join(user_history_10k.set_index('UserID'), on='UserID').fillna('No Experience')

In [11]:
os.makedirs('data/processed')
users_10k.to_csv('data/processed/users.csv', index=False)
jobs_10k.to_csv('data/processed/jobs.csv', index=False)
apps_10k.to_csv('data/processed/applications.csv', index=False)